In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
import glob
from pathlib import Path
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


plan:
1. create data loader shuffeling filenames
2. navie net
3. rnn
4. lstm

from pathlib import Path
result = list(Path(PATH).glob('**/*.txt'))helper functions

In [3]:
def train_and_test_files_paths(path:str=os.getcwd(),test_ratio:float=0.004)->list:

    files = [file_name for file_name in Path(os.getcwd()).rglob("*.gzip")]
    shuffle(files)
    first_train = int(test_ratio * len(files))
    test = files[:first_train]
    train = files[first_train:]
    return train,test

def df_to_X_y(path):
    df = pd.read_parquet(path)
    y_column = df.columns[-1]
    X_columns = [col for col in df.columns if not col == y_column]
    X = torch.Tensor(df[X_columns].values)
    y = torch.from_numpy(df[y_column].values)
    y = y.type(torch.LongTensor)
    return X,y

class parquet_loader():
    def __init__(self,files):
        self.file_names = files
    
    def shuffle_inputs(self):
        shuffle(self.file_names)
    
    def __iter__(self):
        self.shuffle_inputs()
        for file in self.file_names:
            yield df_to_X_y(file)
    
    def __len__(self):
        return len(self.file_names)

In [7]:
train,test = train_and_test_files_paths()
train_loader = parquet_loader(train)
test_loader = parquet_loader(test)

In [8]:
for i in (train_loader,test_loader):
    print(len(i))

79722
320


In [9]:
input_size = 26
number_of_classes = 2
hidden_size = input_size * 2
epocs = 1

In [10]:
class NaiveNet(nn.Module):
    
    def __init__(self,input_size,hidden_size,number_of_classes):
        super(NaiveNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, number_of_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = NaiveNet(input_size,hidden_size,number_of_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

losses =[]
#training loop
files_passed=0
for epoch in range(epocs):
    for X,y in train_loader:
        X=X.to(device)
        y=y.to(device)
        
        #forword
        output = model(X)
        loss = criterion(output,y)
        
        #backwords
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
        
        files_passed+=1
        if files_passed%1_000==0:
            print(f'{round(files_passed/len(train_loader),2)} passed curent batch loss is {loss}')
            
            #calculate accuracy
            correct = total = 0
            with torch.no_grad():
                for X,y in test_loader:
                    X=X.to(device)
                    y=y.to(device)

                    #forword
                    outputs = model(X)
                    _,predictions = torch.max(outputs,1)
                    total += predictions.shape[0]
                    correct += (predictions==y).sum().item()
            accuracy = 100 * correct/total
            print(f'test accuracy = {round(accuracy,3)}')

0.01 passed curent batch loss is 0.49713853001594543
test accuracy = 76.479
0.03 passed curent batch loss is 0.451604425907135
test accuracy = 76.655
0.04 passed curent batch loss is 0.23109152913093567
test accuracy = 79.169
0.05 passed curent batch loss is 0.47540560364723206
test accuracy = 80.99
0.06 passed curent batch loss is 0.43140000104904175
test accuracy = 82.037
0.08 passed curent batch loss is 0.6721282601356506
test accuracy = 81.718
0.09 passed curent batch loss is 0.315256804227829
test accuracy = 82.834
0.1 passed curent batch loss is 0.31350836157798767
test accuracy = 81.83
0.11 passed curent batch loss is 0.3185402750968933
test accuracy = 83.707
0.13 passed curent batch loss is 0.20888172090053558
test accuracy = 84.635
0.14 passed curent batch loss is 0.3447035551071167
test accuracy = 85.27
0.15 passed curent batch loss is 0.6459122896194458
test accuracy = 81.646
0.16 passed curent batch loss is 0.27777522802352905
test accuracy = 85.837
0.18 passed curent batch

somthing is fishy. lets modify our data loader to return filename as well, and see what is going on with test set.

In [4]:
class parquet_loader():
    def __init__(self,files):
        self.file_names = files
    
    def shuffle_inputs(self):
        shuffle(self.file_names)
    
    def __iter__(self):
        self.shuffle_inputs()
        for file in self.file_names:
            x,y = df_to_X_y(file)
            yield x,y,file
    
    def __len__(self):
        return len(self.file_names)
    
def get_original_name(file_name:str):
    return file_name.parts[-1]
    
def get_all_files():
    path = os.getcwd()
    files = [file_name for file_name in Path(os.getcwd()).rglob("*.gzip")]
    return files

def split_to_train_test(files,test_ratio):
    distinct_files = list({get_original_name(file) for file in files})
    print(len(distinct_files))
    shuffle(distinct_files)
    first_train = int(test_ratio * len(distinct_files))
    test_names = set(files[:first_train])
    train = test = list()
    for file in files:
        if get_original_name(file) in test_names:
            test.append(file)
        else:
            train.append(file)
    return train,test

In [6]:
files = get_all_files()
train,test = split_to_train_test(files,0.05)
train_loader = parquet_loader(train)
test_loader = parquet_loader(test)

2582


In [8]:
f = pd.read_parquet(files[0])
f.shape

(2003, 27)

In [11]:
f[f.columns[-1]].sum().astype(int)

870

In [5]:
def get_noise_type_and_snr(file):
    snr = ''.join(charecter for charecter in file.parts[-2] if charecter.isnumeric())
    if snr=='':
        noise_type = 'Quiet'
        snr=0
    elif snr[:2] == '80':
        noise_type = 'car80'
        snr=snr[2:]
    else:
        noise_type = file.parts[-3]
    return noise_type,snr

In [23]:
def train_test_validate_split():
    raw_file_names = get_all_files()
    snr_list ,noise_list , number_of_rows , number_of_talk = [],[],[],[]
    for raw_file in raw_file_names:
        
        noise,snr = get_noise_type_and_snr(raw_file)
        noise_list.append(noise)
        snr_list.append(snr)
        
        df = pd.read_parquet(raw_file)      
        n_rows = df.shape[0]
        number_of_talk_rows = df[df.columns[-1]].sum().astype(int)
        
        number_of_rows.append(n_rows)
        number_of_talk.append(number_of_talk_rows)
        
    files_df = pd.DataFrame({"path": raw_file_names
                            ,"noise": noise_list
                            ,"snr": snr_list
                            ,"rows": number_of_rows
                            ,"rows_of_speech": number_of_talk
                            })
    return files_df

In [25]:
aggregative_df = train_test_validate_split()
aggregative_df.head()

,path,noise,snr,rows,rows_of_speech
0,/home/ido/data/idc/deep learning/vad/Parquet_D...,wind,6,2003,870
1,/home/ido/data/idc/deep learning/vad/Parquet_D...,wind,6,249,160
2,/home/ido/data/idc/deep learning/vad/Parquet_D...,wind,6,308,189
3,/home/ido/data/idc/deep learning/vad/Parquet_D...,wind,6,1234,841
4,/home/ido/data/idc/deep learning/vad/Parquet_D...,wind,6,363,183


In [28]:
speech_ratio = aggregative_df['rows_of_speech'].sum() / aggregative_df['rows'].sum()
speech_ratio

0.5923009676492301

In [14]:
class NaiveNet(nn.Module):
    
    def __init__(self,input_size,hidden_size,number_of_classes):
        super(NaiveNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, number_of_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = NaiveNet(input_size,hidden_size,number_of_classes).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

losses =[]
#training loop
files_passed=0
for epoch in range(epocs):
    for X,y,f in train_loader:
        X=X.to(device)
        y=y.to(device)
        
        #forword
        output = model(X)
        loss = criterion(output,y)
        
        #backwords
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
        
        files_passed+=1
        if files_passed%5_000==0:
            print(f'{round(files_passed/len(train_loader),2)} passed curent batch loss is {loss}')
            
            #calculate accuracy
            correct = total = 0
            with torch.no_grad():
                for X,y,f in test_loader:
                    X=X.to(device)
                    y=y.to(device)

                    #forword
                    outputs = model(X)
                    _,predictions = torch.max(outputs,1)
                    total += predictions.shape[0]
                    correct += (predictions==y).sum().item()
            accuracy = 100 * correct/total
            print(f'test accuracy = {round(accuracy,3)}')

0.06 passed curent batch loss is 0.2591919004917145
test accuracy = 81.4
0.12 passed curent batch loss is 0.31278514862060547
test accuracy = 82.831
0.19 passed curent batch loss is 0.17977643013000488
test accuracy = 87.016
0.25 passed curent batch loss is 0.36816683411598206
test accuracy = 84.622
0.31 passed curent batch loss is 0.12380395084619522
test accuracy = 86.623
0.37 passed curent batch loss is 0.19928976893424988
test accuracy = 87.113
0.44 passed curent batch loss is 0.2964414358139038
test accuracy = 88.766
0.5 passed curent batch loss is 0.44624894857406616
test accuracy = 88.636
0.56 passed curent batch loss is 0.47731178998947144
test accuracy = 88.036
0.62 passed curent batch loss is 0.21565096080303192
test accuracy = 87.795
0.69 passed curent batch loss is 0.1772223263978958
test accuracy = 88.885
0.75 passed curent batch loss is 0.17070409655570984
test accuracy = 89.084
0.81 passed curent batch loss is 0.32416054606437683
test accuracy = 88.763
0.87 passed curent

In [31]:
import seaborn as sns

In [39]:
losses[0].to('cpu')

RuntimeError: CUDA error: unspecified launch failure

In [34]:
loss_df = pd.DataFrame({'loss':losses})

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [33]:
len(losses)
sns.lineplot( x=losses, y=list(range(len(losses))))

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [16]:
for X,y,f in train_loader:
    noise,snr = get_noise_type_and_snr(f)
    print(noise,snr)
    break

white 18


In [20]:
results_dict = {column:[] for column in ('score,noise,snr'.split(','))}
for i,j in zip(results_dict,(1,2,3)):
    print(i,j)

score 1
noise 2
snr 3


In [28]:
results_dict = {column:[] for column in ('accuracy,noise,snr'.split(','))}
with torch.no_grad():
                for X,y,f in test_loader:
                    X=X.to(device)
                    y=y.to(device)
                    outputs = model(X)
                    _,predictions = torch.max(outputs,1)
                    total = predictions.shape[0]
                    correct = (predictions==y).sum().item()
                    accuracy = 100 * correct/total
                    noise,snr = get_noise_type_and_snr(f)
                    for column,value in zip(results_dict,(accuracy,noise,snr)):
                        results_dict[column].append(value)

RuntimeError: CUDA error: unspecified launch failure

In [24]:
results_df = pd.DataFrame(results_dict)
results_df.sort_values(by='accuracy',inplace=True)
results_df.head()

,accuracy,noise,snr
25151,7.812500,white,18
68944,17.894737,white,18
2752,18.045113,white,18
53924,19.685039,white,18
49812,20.930233,white,18


In [25]:
results_df.tail()

,accuracy,noise,snr
53279,100.0,white,18
66108,100.0,white,18
68971,100.0,white,18
16676,100.0,white,18
870,100.0,white,18


In [27]:
results_df.to_csv('first_results.csv',index=False)

todo:<div>
    see on which data preformence are worst<div>
    early stop if accurecy isnt beating best for 10 loops<div>
    show that net without dirivatives is worset